In [ ]:
#### Prepare historical lags item category
if False:    
    # Create "grid" with columns
    index_cols = ['shop_id', 'item_category_id', 'date_block_num']
    
    sales = pd.merge(df_all, df_items[['item_id', 'item_category_id']], on='item_id', how='left')
    
    # For every month we create a grid from all shops/items combinations from that month
    grid = [] 
    for block_num in sales['date_block_num'].unique():
        cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
        cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_category_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    
    # Turn the grid into a dataframe
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
    
    # Groupby data to get shop-item_category-month aggregates
    gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day': 'shop_item_cat_date_sum'}, inplace=True)
    # Join it to the grid
    df_all_grouped = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)
    
    # Groupby data to get shop-month aggregates
    gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day': 'shop_date_sum'}, inplace=True)
    df_all_grouped = pd.merge(df_all_grouped, gb[['shop_id', 'date_block_num', 'shop_date_sum']], 
                              how='left', on=['shop_id', 'date_block_num']).fillna(0)
    
    # Groupby data to get item_category-month aggregates
    gb = sales.groupby(['item_category_id', 'date_block_num'], as_index=False).agg({'item_cnt_day':'sum'})
    gb.rename(columns={'item_cnt_day': 'item_cat_date_sum'}, inplace=True)
    df_all_grouped = pd.merge(df_all_grouped, gb[['item_category_id', 'date_block_num', 'item_cat_date_sum']], 
                              how='left', on=['item_category_id', 'date_block_num']).fillna(0)
    
    # TODO create item_cnt_day_by_item_cat weighted by items count in group
    
    
    # List of columns that we will use to create lags
    cols_to_rename = list(df_all_grouped.columns.difference(index_cols)) 
    
    shift_range = [1, 2, 3, 4, 5, 12]
    
    for month_shift in tqdm.tqdm(shift_range):
        train_shift = df_all_grouped[index_cols + cols_to_rename].copy()
        
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
        
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
        train_shift = train_shift.rename(columns=foo)
    
        df_all_grouped = pd.merge(df_all_grouped, train_shift, on=index_cols, how='left').fillna(0)
    
    # we may have only historical data for target month, we don't know aggregations for it, hence deleting
    df_all_grouped.drop(columns=['shop_item_cat_date_sum', 'item_cat_date_sum', 'shop_date_sum'], inplace=True)
    
    # merge history back to train/test set 
    df_all_gr_by_item_cat = pd.merge(sales, df_all_grouped, on=index_cols, how='left').fillna(0)
    df_all_gr_by_item_cat.drop(columns=['date', 'item_price', 'item_id', 'item_cnt_day'], inplace=True)
    df_all_gr_by_item_cat = downcast_dtypes(df_all_gr_by_item_cat)
    
    del grid, df_all_grouped, sales, train_shift
    gc.collect();

In [ ]:
from scipy.stats import randint as sp_randint

lgb_params = {
               'feature_fraction': [0.55, 0.65, 0.75, 0.85, 0.95],
               'metric': ['rmse'],
               'nthread':[-1], 
               'min_data_in_leaf': [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10], 
               'bagging_fraction': [0.65, 0.75, 0.85], 
               'learning_rate': [0.1, 0.01, 0.03], 
               'objective': ['mse'], 
               'bagging_seed': [2**7], 
               'num_leaves': sp_randint(100, 1000),
               'bagging_freq':[1],
               'verbose':[0] 
              }
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'mse', 
            "eval_set" : [(cv_X_test, cv_Y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}

n_HP_points_to_test = 100

clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=10)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=lgb_params, 
    n_iter=n_HP_points_to_test,
    scoring='r2',
    cv=4,
    refit=True,
    random_state=314,
    verbose=False)
gs.fit(cv_X_train, cv_Y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

lgb_params = {
               'feature_fraction': 0.95,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 16, 
               'bagging_fraction': 0.85, 
               'learning_rate': 0.1, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 906,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=Y_train), 100)
pred_lgb = model.predict(X_test)
get_result_df(pred_lgb).to_csv('result_lgb_26_features.csv', index=False)
# print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))
# Your solution can be improved by a lot! Your public and private LB scores are: 1.319619 and 1.318699.

In [ ]:
def split_train_and_postprocess_for_linear_bk(df_all_data):
    cat_columns = ['item_id', 'shop_id', 'item_category_id', 'type_code', 'subtype_code','city_code']
    for cat_column in cat_columns:
        df_all_data[cat_column] = df_all_data[cat_column].astype('category')
            
    cats_date = cat_columns + ['date_block_num']
    df_cat = df_all_data[cats_date]
    df_scale = df_all_data[list(set(df_all_data.columns) - set(cat_columns))]
      
    scaler = StandardScaler()
    scaler.fit(trim_12(df_scale).drop(columns=['target']))
    
    encoder = OneHotEncoder(drop='first')
    encoder.fit(trim_12(df_cat)[cat_columns])
    
    train_cat, X_test_cat = divide_df(df_cat, TRAIN_SIZE)
    train_cat = trim_12(train_cat)
    X_test_cat = trim_12(X_test_cat).drop(columns=['date_block_num'])
    
    max_date = train_cat['date_block_num'].max()
               
    cv_X_test_cat = train_cat[train_cat['date_block_num'] == max_date].drop(columns=['date_block_num'])
    cv_X_train_cat = train_cat[train_cat['date_block_num'] < max_date].drop(columns=['date_block_num'])
    
    train_scale, X_test_scale = divide_df(df_scale, TRAIN_SIZE)
    train_scale = trim_12(train_scale)
    X_test_scale = trim_12(X_test_scale)
    X_train_scale = train_scale.drop(columns=['target'])
    Y_train = train_scale['target']
    
    cv_X_test_scale = X_train_scale[X_train_scale['date_block_num'] == max_date]
    cv_Y_test = Y_train[X_train_scale['date_block_num'] == max_date]

    cv_Y_train = Y_train[X_train_scale['date_block_num'] < max_date]
    cv_X_train_scale = X_train_scale[X_train_scale['date_block_num'] < max_date]
               
    # scaling
    X_test_scale = scaler.transform(X_test_scale)
    cv_X_test_scale = scaler.transform(cv_X_test_scale)
    cv_X_train_scale = scaler.transform(cv_X_train_scale)
    
    X_test_cat = encoder.transform(X_test_cat)
    X_test_scale = csr_matrix(X_test_scale)
    X_test = hstack((X_test_cat, X_test_scale))
               
    cv_X_test_cat = encoder.transform(cv_X_test_cat)
    cv_X_test_scale = csr_matrix(cv_X_test_scale)
    cv_X_test = hstack((cv_X_test_cat, cv_X_test_scale))
    
    cv_X_train_cat = encoder.transform(cv_X_train_cat)    
    cv_X_train_scale = csr_matrix(cv_X_train_scale)
    cv_X_train = hstack((cv_X_train_cat, cv_X_train_scale))
               
    return cv_X_train, cv_Y_train, cv_X_test, cv_Y_test,X_test 

In [ ]:
all_data = pd.merge(all_data, encode_target_by_category(sales, 'shop_id'), 
                        how='left', 
                        on=['shop_id', 'date_block_num']).fillna(0)

def encode_target_by_category(df, cat_column):
    df = df[[cat_column, 'date_block_num', 'item_cnt_day']]
    ind_cols = [cat_column, 'date_block_num']
    summ = df.groupby(ind_cols, as_index=False)['item_cnt_day'].sum()
    print(summ)
    summ[cat_column + '_target_enc'] = summ.groupby([cat_column], as_index=False)\
        .apply(lambda  x: x.sort_values(by='date_block_num')['item_cnt_day'].cumsum()).reset_index(drop=True)
    summ['cumcount'] = summ.groupby(cat_column).cumcount()
    
    print(summ)
    
    summ[cat_column + '_target_enc'] = summ[cat_column + '_target_enc'] - summ['item_cnt_day']
    summ[cat_column + '_target_enc'] = summ[cat_column + '_target_enc'] / summ['cumcount']
    print(summ.head(50))
    return summ.drop(columns = ['item_cnt_day', 'cumcount'])